## Creating run_query(), run_command(),  and show_tables() functions

In [92]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

%matplotlib inline


def run_query(q):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql_query(q,conn)
    
def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(c)
def show_tables():
    q = """ SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");
"""
    return run_query(q)

# Running show_tables() function to return list of tables and
# views in database

print (show_tables())
show_tables()
    
    

              name   type
0            album  table
1           artist  table
2         customer  table
3         employee  table
4            genre  table
5          invoice  table
6     invoice_line  table
7       media_type  table
8         playlist  table
9   playlist_track  table
10           track  table


,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


## Writing query that returns number of tracks old for each genre

In [96]:
q = """


WITH
    customers AS
        (SELECT * FROM invoice i
        GROUP BY 1
        ),
    inln AS
        (SELECT * FROM invoice_line il
        INNER JOIN customers c ON c.invoice_id = il.invoice_id
        ),
    tracks AS
        (SELECT * FROM genre g
        INNER JOIN track tr ON g.genre_id = tr.genre_id
        )
    
SELECT name genre, SUM(total) totals FROM tracks trs
INNER JOIN inln ON inln.track_id = trs.track_id
GROUP BY 1
ORDER BY 2 DESC;
    """
               

run_query(q)

,genre,totals
0,Rock,26751.78
1,Metal,5316.30
2,Alternative & Punk,4841.10
3,R&B/Soul,1751.31
4,Latin,1705.77
5,Blues,1379.07
6,Jazz,1302.84
7,Alternative,1095.93
8,Easy Listening,951.39
9,Electronica/Dance,614.79
